<a href="https://colab.research.google.com/github/zhestyatsky/abbyy-nlp-course/blob/main/sem4/hw4_cnn_rnn_for_clf_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1
Сделайте char-level CNN для решения этой же задачи

In [14]:
%%writefile requirements.txt
pytorch-lightning
scikit-learn
allennlp

Overwriting requirements.txt


In [15]:
!pip install --upgrade -r requirements.txt

Requirement already up-to-date: pytorch-lightning in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 1)) (1.1.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (0.23.2)
     |████████████████████████████████| 512kB 7.4MB/s 
     |████████████████████████████████| 133kB 13.6MB/s 
     |████████████████████████████████| 1.1MB 14.5MB/s 
     |████████████████████████████████| 317kB 27.7MB/s 
     |████████████████████████████████| 1.4MB 29.6MB/s 
     |████████████████████████████████| 266kB 55.0MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 7.1MB 19.3MB/s 
  Created wheel for overrides: filename=overrides-3.1.0-cp36-none-any.whl size=10175 sha256=5463340bdc1a7d19a1562dcbf5981c3c1ad7c246c4a6aeb28e8eb46ce652c1eb
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147c95e595f1289a86a3cc82ed65df57582c65a9
  Created wheel for js

## Датасет

In [3]:
!rm negative.csv
!rm positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

rm: cannot remove 'negative.csv': No such file or directory
rm: cannot remove 'positive.csv': No such file or directory
--2020-12-20 19:02:40--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2020-12-20 19:02:40--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc908525b40789e479e7716808c3.dl.dropboxusercontent.com/cd/0/inline/BFe5jrQBun7zC_oyT502acRg5W9KqmRQ1FuTHDArB1UmX_lWsRofuvHLM0xlgMU1yvuDGw-f5UNJ418RwMIZAHc_KGvHvL3M6_swjKna-eos-g/file# [following]
--2020-12-20 19:02:41--  https://uc908525b40789e479e7716808c3.dl.dropboxusercontent.com/cd/0/inline/BFe5jrQBun7z

In [4]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size], data_negative['text'].values[:sample_size]), axis=0)

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

df_train = pd.DataFrame(columns=['text', 'label'])
df_val = pd.DataFrame(columns=['text', 'label'])
df_test = pd.DataFrame(columns=['text', 'label'])

data = [preprocess_text(t) for t in raw_data]
labels = [1] * sample_size + [0] * sample_size
df_train['text'], df_test['text'], df_train['label'], df_test['label'] = train_test_split(data, labels, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
df_train

,text,label
85913,раньше все встречались у фонтана в гуме а тепе...,1
42792,ни когда не пойму любовь женщины к женщине хот...,1
85556,два сеанса в кино вот что я люблю парам пам па...,1
36360,я нашел мой наряд для кэти перри USER URL via ...,1
154940,USER ууууу всеее развод обидки и все дела,1
...,...,...
73349,закидываю свой вконтакторостер строчками из пе...,1
109259,гребаный понедельник гребанные 4 пары хочу дом...,0
50057,болячки вроде подживают но губы увеличелись в ...,0
5192,USER я знала что она добрая а за что она тебе,1


In [22]:
from allennlp.data.tokenizers import CharacterTokenizer

tokenizer = CharacterTokenizer()

train_labels = df_train["label"].tolist()
val_labels = df_val["label"].tolist()
test_labels = df_test["label"].tolist()

texts_set = set(df_train["text"].tolist()) | set(df_val["text"].tolist()) | set(df_test["text"].tolist())

train_texts = [[token.text for token in tokenizer.tokenize(text)] for text in df_train["text"].tolist()]
val_texts = [[token.text for token in tokenizer.tokenize(text)] for text in df_val["text"].tolist()]
test_texts = [[token.text for token in tokenizer.tokenize(text)] for text in df_test["text"].tolist()]

In [24]:
from collections import Counter


class Vocabulary:
    def __init__(self):
        self.word2index = {
            "<pad>": 0,
            "<unk>": 1
        }
        self.index2word = ["<pad>", "<unk>"]

    def build(self, texts, min_count=7):
        words_counter = Counter(token for tokens in texts for token in tokens)
        for word, count in words_counter.most_common():
            if count >= min_count:
                self.word2index[word] = len(self.word2index)
        self.index2word = [word for word, _ in sorted(self.word2index.items(), key=lambda x: x[1])]
    
    def get_pad(self):
        return self.word2index["<pad>"]

    def get_unk(self):
        return self.word2index["<unk>"]
    
    @property
    def size(self):
        return len(self.index2word)
    
    def top(self, n=100):
        return self.index2word[1:n+1]
    
    def get_index(self, word):
        return self.word2index.get(word, self.get_unk())
    
    def get_word(self, index):
        return self.index2word[index]

vocabulary = Vocabulary()
vocabulary.build(train_texts)
assert vocabulary.word2index[vocabulary.index2word[10]] == 10
print(vocabulary.size)
print(vocabulary.top(100))

75
['<unk>', ' ', 'о', 'а', 'е', 'т', 'н', 'и', 'с', 'л', 'р', 'в', 'к', 'у', 'д', 'м', 'п', 'я', 'ь', 'б', 'ч', 'г', 'U', 'R', 'ы', 'з', 'S', 'E', 'х', 'й', 'ж', 'ш', 'ю', 'э', 't', 'r', 'щ', 'ц', 'ф', 'L', 'd', 'a', 'o', '1', 'e', 'i', '3', '2', 's', 'l', 'n', 'm', 'p', 'c', '9', '5', 'u', 'f', '4', 'h', 'g', 'b', 'w', 'y', 'k', 'v', '8', '6', '7', 'ъ', 'x', 'j', 'q', 'z']


In [25]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler


class TextDataset(Dataset):
    def __init__(self, tokenized_texts, labels, vocabulary, max_seq_len=200):
        self.samples = []
        assert len(tokenized_texts) != 0
        assert len(tokenized_texts) == len(labels)
        for text, label in zip(tokenized_texts, labels):
            indices = [vocabulary.get_index(word) for word in text][:max_seq_len]
            indices += [vocabulary.get_pad() for _ in range(max_seq_len - len(indices))]
            self.samples.append((torch.LongTensor(indices), torch.FloatTensor([label])))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]


BATCH_SIZE = 128

train_data = TextDataset(train_texts, train_labels, vocabulary)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = TextDataset(val_texts, val_labels, vocabulary)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = TextDataset(test_texts, test_labels, vocabulary)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [26]:
import os
import torch
import random
import numpy as np

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [27]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule
from pytorch_lightning.metrics import Accuracy
 

class SimpleModel(LightningModule):
    def __init__(self, vocab_size, embedding_dim=32):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(vocab_size, embedding_dim)
        self.loss = nn.BCEWithLogitsLoss()
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
    
    def forward(self, inputs, labels):
        raise NotImplementedError("forward not implemented")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.valid_accuracy.update(logits, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.test_accuracy.update(logits, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)

In [29]:
import torch
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping  

class CnnModel(SimpleModel):
    def __init__(self, vocab_size, embedding_dim=128, filters=((2, 10), (3, 8))):
        super().__init__(vocab_size, embedding_dim)
        
        self.filters = []
        all_filters_count = 0
        for kernel_size, filters_count in filters:
            all_filters_count += filters_count
            self.filters.append(nn.Conv2d(1, filters_count, (kernel_size, embedding_dim), padding=(1, 0)))
        self.filters = nn.ModuleList(self.filters)
        self.relu_layer = nn.ReLU()
        self.dropout_layer = nn.Dropout(0.2)
        self.out_layer = nn.Linear(all_filters_count, 1)
    
    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        # batch_size x num_words x embedding_dim
        projections = projections.unsqueeze(1)
        # batch_size x 1 x num_words x embedding_dim
        results = []
        for f in self.filters:
            convolved = self.dropout_layer(self.relu_layer(f(projections))).squeeze(3)
            # batch_size x num_filters x num_words
            pooling = torch.max(convolved, 2)[0]
            # batch_size x num_filters
            results.append(pooling)
        output = torch.cat(results, 1)
        # batch_sisze x sum(num_filters)
        # 128 x 18
        logits = self.out_layer.forward(output)
        loss = self.loss(logits, labels)
        return loss, logits

cnn_model = CnnModel(vocabulary.size)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(cnn_model, train_loader, val_loader)
trainer.test(cnn_model, test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params
-------------------------------------------------------
0 | embeddings_layer | Embedding         | 9.6 K 
1 | loss             | BCEWithLogitsLoss | 0     
2 | valid_accuracy   | Accuracy          | 0     
3 | test_accuracy    | Accuracy          | 0     
4 | filters          | ModuleList        | 5.7 K 
5 | relu_layer       | ReLU              | 0     
6 | dropout_layer    | Dropout           | 0     
7 | out_layer        | Linear            | 19    
-------------------------------------------------------
15.3 K    Trainable params
0         Non-trainable para


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.6960, device='cuda:0'),
 'test_acc_epoch': tensor(0.6960, device='cuda:0'),
 'test_loss': tensor(0.5726, device='cuda:0'),
 'val_acc': tensor(0.6977, device='cuda:0'),
 'val_acc_epoch': tensor(0.6977, device='cuda:0'),
 'val_loss': tensor(0.5733, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.6960464715957642,
  'test_acc_epoch': 0.6960464715957642,
  'test_loss': 0.5726295709609985,
  'val_acc': 0.6976770162582397,
  'val_acc_epoch': 0.6976770162582397,
  'val_loss': 0.5732998251914978}]